# 확률적 경사하강법(SGD)
- P와 Q행렬로 계산된 예측 R행렬 값이 실제 R행렬 값과 가장 최소의 오류 값을 가지도록 반복적인 비용함수 최적화를 통해 P와 Q를 유추해내는 것
    1. P와 Q를 임의의 값을 가진 행렬로 설정
    2. P와 Q.T를 곱해 예측 R행렬을 만들고 예측 R행렬과 실제 R행렬에 해당하는 오류 값 계산
    3. 이 오류 값을 최소화할 수 있도록 P와 Q행렬을 적절한 값으로 업데이트
    4. 만족할 만한 오류 값을 가질 때까지 2,3번을 반복하면서 P와 Q값을 업데이트해 근사화함 

- 실제값과 예측값의 오류 최소화와 L2규제를 고려한 비용 함수식은 다음과 같음

<img src = 'https://blog.kakaocdn.net/dn/bHGFQX/btq1XgAF3Ph/4jyakP1KPgLPuoznV93cHk/img.png' width = 400px>

- 일반적으로 사용자-아이템 평점 행렬의 경우 행렬 분해를 위해 단순히 예측 오류값의 최소화와 학습시 과적합을 피하기 위해 규제를 반영한 비용함수를 적용함
- 그리고 위와 같은 비용함수를 최소화하기 위해 새롭게 업데이트 되는 p'와 q'는 다음과 같이 계산할 수 있음
<img src = 'https://blog.kakaocdn.net/dn/ebxmro/btq1XMF8bKa/QLzPYJwM78gwnUcMAPrDlk/img.png' width = 500px>

## 직접 코드 짜보기

In [1]:
import numpy as np

In [37]:
# 사용자 - 아이템 매트릭스
R = np.array([[4, np.nan, np.nan, 2, np.nan],
              [np.nan, 5, np.nan, 3, 1],
              [np.nan, np.nan, 3, 4, 4],
              [5, 2, 1, 2, np.nan]])

# 행: 사용자수, 열: 아이템 수
num_users, num_iters = R.shape

# 잠재요인 차원 수
k = 3

np.random.seed(1)

# P와 Q행렬 크기 지정하고 정규분포를 따르는 임의의 값으로 입력
P = np.random.normal(scale = 1./k, size = (num_users, k))
Q = np.random.normal(scale = 1./k, size = (num_iters, k))

In [38]:
P, Q

(array([[ 0.54144845, -0.2039188 , -0.17605725],
        [-0.35765621,  0.28846921, -0.76717957],
        [ 0.58160392, -0.25373563,  0.10634637],
        [-0.08312346,  0.48736931, -0.68671357]]),
 array([[-0.1074724 , -0.12801812,  0.37792315],
        [-0.36663042, -0.05747607, -0.29261947],
        [ 0.01407125,  0.19427174, -0.36687306],
        [ 0.38157457,  0.30053024,  0.16749811],
        [ 0.30028532, -0.22790929, -0.04096341]]))

In [39]:
P.shape, Q.T.shape

((4, 3), (3, 5))

In [40]:
from sklearn.metrics import mean_squared_error

In [52]:
# 실제 행렬 R과 예측 행렬 R의 오차를 계산해주는 사용자함수 생성

def get_rmse(R, P, Q, non_zeros, verbose = False):
    error = 0
    
    # 예측행렬 R
    full_pred_matrix = np.dot(P, Q.T)
    if verbose:
        print(full_pred_matrix)
    
    # 실제 R 행렬에서 null이 아닌 값의 위치 인덱스 추출하여 rmse 추출
    x_non_zero_idx = [idx[0] for idx in non_zeros]
    y_non_zero_idx = [idx[1] for idx in non_zeros]
    
    non_zero_R = R[x_non_zero_idx, y_non_zero_idx]

    non_zero_pred_matrix = full_pred_matrix[x_non_zero_idx, y_non_zero_idx]

    mse = mean_squared_error(non_zero_R, non_zero_pred_matrix)
    rmse = np.sqrt(mse)
    
    return rmse

In [53]:
# 확률적 경사하강법(SGD) 기반 행렬 분해 수행
non_zeros = [(i, j, R[i, j])for i in range(num_users) for j in range(num_iters) if R[i,j] >0 ]

steps = 1000
learning_rate = 0.01,
r_lambda = 0.01

# SGD 기법으로 P와 Q의 매트릭스 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        eij = R[i, j] - np.dot(P[i,:], Q[j, :].T)
        
        # l2정규화를 반영한 업데이트
        P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])
        
    
    if step % 50 == 0:
        rmse = get_rmse(R, P, Q, non_zeros, verbose = True)
        print(f'####{step}번째 업데이트 중 ### \n\t RMSE: {rmse:.4f}')
    else:
        rmse = get_rmse(R, P, Q, non_zeros)

[[3.99126388 1.66270017 1.16675409 1.99893824 1.61144893]
 [5.25847427 4.977519   0.87738292 2.98549131 1.00391285]
 [5.73126177 1.74952726 2.98766441 3.97918784 3.98527112]
 [4.97197856 2.00367238 1.00479425 2.00565618 1.4683065 ]]
####0번째 업데이트 중 ### 
	 RMSE: 0.0143
[[3.99126438 1.66798136 1.16557757 1.99892074 1.61089476]
 [5.24432887 4.9775228  0.87877246 2.98551786 1.00392663]
 [5.72190277 1.76016735 2.987675   3.97920333 3.98526228]
 [4.97200771 2.00364585 1.00476901 2.00558989 1.47040713]]
####50번째 업데이트 중 ### 
	 RMSE: 0.0143
[[3.99126492 1.67319078 1.16441697 1.99890329 1.61034369]
 [5.23027679 4.97752661 0.88019254 2.98554403 1.00394036]
 [5.71259816 1.77071353 2.9876855  3.97921869 3.98525348]
 [4.97203645 2.00361942 1.00474399 2.0055246  1.47247104]]
####100번째 업데이트 중 ### 
	 RMSE: 0.0143
[[3.9912655  1.67832937 1.1632722  1.99888589 1.60979572]
 [5.21631757 4.97753043 0.88164223 2.98556982 1.00395406]
 [5.70334774 1.7811671  2.98769593 3.97923391 3.98524472]
 [4.97206479 2.0035

In [57]:
# 예측 행렬 R
print(P, Q)
print()

pred_R = np.dot(P, Q.T)
print(np.round(pred_R, 3))

[[ 0.79071859  0.64234346 -0.86960641]
 [-0.06268751  0.46214559 -2.22594423]
 [ 2.07736263 -0.08253501 -1.35685766]
 [ 0.77646674  1.18880589 -0.88452539]] [[ 1.49365430e+00  1.78223589e+00 -1.91513467e+00]
 [-4.48626365e-01  3.82755467e-01 -2.14407133e+00]
 [ 1.10810396e+00 -2.44022592e-01 -4.90682416e-01]
 [ 1.02068846e+00  1.17698968e-05 -1.37017649e+00]
 [ 1.56831721e+00 -1.51265697e-01 -5.26698048e-01]]

[[3.991 1.756 1.146 1.999 1.601]
 [4.993 4.978 0.91  2.986 1.004]
 [5.554 1.946 2.988 3.979 3.985]
 [4.972 2.003 1.004 2.005 1.504]]


```
R = np.array([[4, np.nan, np.nan, 2, np.nan],
              [np.nan, 5, np.nan, 3, 1],
              [np.nan, np.nan, 3, 4, 4],
              [5, 2, 1, 2, np.nan]])
```         
실제R행렬과 예측행렬 R의 값이 그렇게 크게 차이나지 않음